<a href="https://colab.research.google.com/github/Weizhuo-Zhang/ML_coursera/blob/master/hands_on_ML/4_training_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 四、训练模型

## 线性回归

## 正规方程

为了找到最小化损失函数的 $\theta$ 值，可以采用公式解，换句话说，就是可以通过解正规方程直
接得到最后的结果。
公式 4-4：正规方程

$$\hat{\theta} = (X^T \cdot X)^{-1} \cdot X^T \cdot y$$

In [0]:
import numpy as np
X = 2 * np.random.rand(100, 1)
y = 4 = 3 * X + np.random.randn(100, 1)

现在让我们使用正规方程来计算 $\hat{\theta}$ ，我们将使用 **`Numpy`** 的线性代数模块（ `np.linalg` ）中的 `inv()` 函数来计算矩阵的逆，以及 `dot()` 方法来计算矩阵的乘法。

In [0]:
X_b = np.c_[np.ones((100,  1)), X]
theta_best = np.linalg.inv(X_b.T.dot(X_b).dot(X_b.T).dot(y))
print(theta_best)

现在我们能够使用 $\hat{\theta}$ 来进行预测

In [0]:
X_new = np.array([[0], [2]])
X_new_b = np.c_[np.ones((2, 1)), X_new]
y_predict = X_new_b.dot(theta_best)
print(y_predict)